In [4]:
import tkinter as tk
from tkinter import *
from tkinter import ttk
from tkinter.messagebox import showinfo
from tkinter import filedialog as fd
from PIL import ImageTk,Image  
import PIL

from collections import Counter
import numpy as np
from pylab import *
import matplotlib.pyplot as plt
import math
import struct
import os
import heapq
from scipy import signal as sig


In [5]:
### Reading and Writing PGM Files (Greyscale Images) ###

def readpgm2(name):
# Reads a pgm file (ASCII/P2) 
# Returns a 2D array of ints with a header of metada
# Returns [[w,h], [gray], data]

    with open(name) as f:
        lines = f.readlines()
    for l in list(lines):
        if l[0] == '#':
            lines.remove(l)
    assert lines[0].strip() == 'P2' 
    arr = []
    for li in lines[1:]:
        s_li = li.split()
        i_li = list(map(lambda x: int(x), s_li))
        arr.append(i_li)
    return arr

def readpgm5(name):
# Reads a pgm file (binary/P5)
# Returns a 2D array of ints with a header of metada
# Returns [[w,h], [gray], data]

    f = open(name, "rb")
    header = []
    w=0
    h=0
    while(len(header)<3):
        temp = f.readline()
        if temp[0]==35:
            print("Comment detected in " + name)
        elif len(header)==0:
            assert (temp == b'P5\n')
            header.append(temp.decode("utf-8"))
        elif len(header)==1:
            (w, h) = [int(i.decode("utf-8")) for i in temp.split()]
            header.append([w,h])
        elif len(header)==2:
            header.append([int(temp)])

    data = []
    data.append(header[1])
    data.append(header[2])
    for y in range(h):
        row = []
        for y in range(w):
            row.append(ord(f.read(1)))
        data.append(row)
    return data

def readpgm(name):
# Reads a pgm file (binary or ascii)
# Returns a 2D array of ints with a header of metada
# Returns [[w,h], [gray], data]

    try:
        try:
            return readpgm5(name)
        except:
            return readpgm2(name)
    except:
        print("Error reading " + name)
        return

def writepgm(file_name, data):
# Writes a pgm file (binary/P5) given pgm data with header

    file_handle = open (file_name, 'wb')
    w = data[0][0]
    h = data[0][1]
    gray = data [1][0]
    pgm_header = f'P5\n{w} {h}\n{gray}\n'
    
    file_handle.write (bytearray (pgm_header, 'ascii')) 

    grayV = np.reshape (data[2:], w*h)

    grayB = struct.pack ('%sB' % len(grayV), *grayV)
    file_handle.write(grayB)
    file_handle.close()
    return


In [6]:
### Reading and Writing PPM Files (Full-Color Images) ###

def readppm3(name):
# Reads a ppm file (ASCII/P3) 
# Returns a 2D array of ints with a header of metada
# Returns [[w,h], [gray], data]

    with open(name) as f:
        lines = f.readlines()
    for l in list(lines):
        if l[0] == '#':
            lines.remove(l)
    assert lines[0].strip() == 'P3' 
    arr = []
    for li in lines[1:]:
        s_li = li.split()
        i_li = list(map(lambda x: int(x), s_li))
        arr.append(i_li)
    return arr

def readppm6(name):
# Reads a pgm file (binary/P6)
# Returns a 2D array of ints with a header of metada
# Returns [[w,h], [gray], data]

    f = open(name, "rb")
    header = []
    w=0
    h=0
    while(len(header)<3):
        temp = f.readline()
        if temp[0]==35:
            print("Comment detected in " + name)
        elif len(header)==0:
            assert (temp == b'P6\n')
            header.append(temp.decode("utf-8"))
        elif len(header)==1:
            (w, h) = [int(i.decode("utf-8")) for i in temp.split()]
            header.append([w,h])
        elif len(header)==2:
            header.append([int(temp)])

    data = []
    data.append(header[1])
    data.append(header[2])
    for y in range(h):
        row = []
        for y in range(3*w):
            row.append(ord(f.read(1)))
        data.append(row)
    return data

def readppm(name):
# Reads a ppm file (binary or ascii)
# Returns a 2D array of ints with a header of metada
# Returns [[w,h], [gray], data]

    try:
        try:
            return readppm6(name)
        except:
            return readppm3(name)
    except:
        print("Error reading " + name)
        return
    
def writeppm(file_name, data):
# Writes a ppm file (binary/P6) given ppm data with header

    file_handle = open (file_name, 'wb')
    w = data[0][0]
    h = data[0][1]
    gray = data [1][0]
    pgm_header = f'P6\n{w} {h}\n{gray}\n'
    
    file_handle.write (bytearray (pgm_header, 'ascii')) 

    grayV = np.reshape (data[2:], 3*w*h)

    grayB = struct.pack ('%sB' % len(grayV), *grayV)
    file_handle.write(grayB)
    file_handle.close()
    return


In [7]:
# Utility Functions

def convertHSV(r, g, b):
# Converts a color from the RGB color model to the HSV color model
# r, g, b values should be in range [0,255]
# Returns a triple of floats in range [0,1] corresponding to (h,s,v)

    rr = r/255
    gg = g/255
    bb = b/255
    maxx = max(rr,gg,bb)
    minn = min(rr,gg,bb)
    delt = maxx-minn
    
    # hue 
    if delt == 0:
        h = 0
    elif maxx == rr:
        h = (((gg-bb)/delt) % 6) / 6
    elif maxx == gg:
        h = (((bb-rr)/delt) + 2) / 6
    else: # maxx == bb
        h = (((rr-gg)/delt) + 4) / 6

    # saturation
    if maxx == 0:
        s = 0
    else:
        s = delt/maxx
        
    # value
    v = maxx
    
    return (h,s,v)

def convertRGB(h, s, v):
# Converts a color from the HSV color model to the RGB color model
# h, s, v values should be in range [0,1]
# Returns a triple of ints in range [0,255] ; (r,g,b)

    H = h*360
    c = v*s
    abss = ((H/60) % 2) - 1
    x = c * (1 - abs(abss))
    m = v - c
    
    if H < 60:
        (rr,gg,bb) = (c,x,0)
    elif H < 120:
        (rr,gg,bb) = (x,c,0)
    elif H < 180:
        (rr,gg,bb) = (0,c,x)
    elif H < 240:
        (rr,gg,bb) = (0,x,c)
    elif H < 300:
        (rr,gg,bb) = (x,0,c)
    else: # H < 360:
        (rr,gg,bb) = (c,0,x)
        
    r = round((rr+m) * 255)
    g = round((gg+m) * 255)
    b = round((bb+m) * 255)
    
    return (r,g,b)

def padded_data(data, d):
# Pads a matrix with infinities and returns a new 2d array
# d = number infinities before and after each row/col
# data = pgm data including header metadata

    padded = []
    dimX = data[0][0]
    #create row of infinity
    inf = float("inf")
    inf_row = [inf] * (d + dimX + d)
    
    for _ in range(d):
        padded.append(inf_row)
    
    for r in data[2:]:
        row = []
        for _ in range(d):
            row.append(inf)
        for c in r:
            row.append(c)
        for _ in range(d):
            row.append(inf)
        padded.append(row)
        
    for _ in range(d):
        padded.append(inf_row)
        
    return padded

def find_neighbors(p, x, y, n):
# Returns the values of the neighbors of a pixel
# p = padded data
# x,y is the pixel whose neighbors are being calculated
# n is the dimension of the square neighborhood
    
    assert n%2 == 1 and n >= 3
    neighbors = []
    dist = int((n-1)/2)
    for i in range(x-dist, x+dist+1): #range(0, )
        for j in range(y-dist, y+dist+1):
            neighbors.append(p[i][j])
    return neighbors

def min_filt(data, n): 
# Applies a minimum filter onto the image
# data = pgm data including header metadata
# Filter size is nxn
# Returns data of the new greyscale image including metadata

    assert n%2 == 1 and n >= 3
    d = int((n-1)/2)
    p = padded_data(data, d)
    
    new_data = []
    new_data.append(data[0])
    new_data.append(data[1])
    for i,r in enumerate(p[d:(0-d)]):
        row = []
        for j,c in enumerate(r[d:(0-d)]):
            neighbors = find_neighbors(p, i+d, j+d, n)
            minn = min(neighbors)
            row.append(minn)
        new_data.append(row)
    return new_data

def scale_data(data, L):
# Scales data to the range [0,L-1] where L = number of gray levels
# data = pgm data including header metadata
# Returns data of the scaled image including metadata
    #gray = data[1][0]
    minv = min([min(d) for d in data[2:]])
    print("in scale_data: minv = "+str(minv))
    shft = []
    for r in data[2:]:
        row = []
        for c in r:
            row.append(c-minv)
        shft.append(row)
    maxv = max([max(d) for d in shft])
    print("in scale_data: maxv = "+str(maxv))
    fact = L/maxv
    scale = []
    scale.append(data[0])
    scale.append(data[1])
    for r in shft:
        row = []
        for c in r:
            if(L==1):
                row.append(c*fact)
            else:
                row.append(int(c*fact))
        scale.append(row)
    return scale


In [8]:
def deep_points(dmap, p):
# Finds the indices of the deepest values in a depth map
# Deepest points determined by lowest values
# dmap: a 2D array representing the depth pgm of an image, incl. header metadata
# p: a decimal indicating which top percentile of points to return
# e.g. p=0.1 returns the top 0.1% deepest points
# Returns a list of (value, (x,y))

    assert p <= 100
    w = dmap[0][0]
    h = dmap[0][1]
    
    n = w*h # total number of pixels
    pp  = int(n * p / 100) # number of pixels to return
    
    # heapq is by default a min heap
    # since we need a max heap, we negate c upon pushing
    heap = [] 
    for i,r in enumerate(dmap[2:]):
        for j,c in enumerate(r):
            if(len(heap) <= pp):
                heapq.heappush(heap, (c,(i,j)))
            else:
                # heap[0][0] = the most negative # -> biggest for us
                if c > heap[0][0] and c != 0:
                    heapq.heapreplace(heap, (c,(i,j)))
                    
    # again, we use nlargest to find the lowest values 
    # since our values are negative
    points = []
    for (val,point) in heapq.nlargest(pp, heap):
        print(val)
        points.append(point)
    print(points)
    return points
    
def est_atm_light(dmap, data):
# Given an image and its depth map, finds the value of atmospheric light (r,g,b)
# dmap: a 2D array representing the depth pgm of an image, incl. header metadata
# data: a 2D array representing the ppm of an image, incl. header metadata

    deep =  deep_points(dmap, .1)
    maxval = 0
    maxx = (0,0,0)
    for (x,y) in deep:
        r = data[x+2][3*y]
        g = data[x+2][(3*y)+1]
        b = data[x+2][(3*y)+2]
        i = (r+g+b)/3
        
        if i>maxval: 
            maxval = i
            maxx=(r,g,b)
    return maxx

def raw_depth_map(data):
# Finds the raw depth map of the given image
# Does not scale data; 
# data: a 2D array representing the ppm of an image, incl. header metadata
# Returns a 2D array representing the raw depth pgm of an image, incl. header metadata

    w = data[0][0]
    h = data[0][1]
    lvls = data[1][0]

    dmap = []
    dmap.append(data[0])
    dmap.append(data[1])
    
    # linear coeffs from study
    th0 = 0.121779
    th1 = 0.959710
    th2 = -0.780245
    sigma = 0.041337
    
    # random normal distribution
    eps = (np.random.normal(0, sigma, size=(h,w))).tolist()
    
    for r in range(h):
        row = []
        for c in range(w):
            rr = data[r+2][3*c]
            gg = data[r+2][(3*c)+1]
            bb = data[r+2][(3*c)+2]
            (_,s,v) = convertHSV(rr,gg,bb)
            # calculate depth
            d = th0 + (th1 * v) + (th2 * s) + eps[r][c]
            row.append(d)
        dmap.append(row)
    return dmap

def guided_filt(guide, img, r, eps):
# Applies a guided image filter onto the image
# img, guide = pgm data including header metadata
# r = dimension of the guided image filter
# Returns data of the new greyscale image including metadata

# Code taken from https://github.com/jacob5412
# Algorithm from "Guided Image Filtering" by Kaiming He et al

    w = img[0][0]
    h = img[0][1]
    I = np.array(guide[2:])
    P = np.array(img[2:])
    window = np.ones((r, r)) / (r * r)

    meanI = sig.convolve2d(I, window, mode="same")
    meanP = sig.convolve2d(P, window, mode="same")

    corrI = sig.convolve2d(I * I, window, mode="same")
    corrIP = sig.convolve2d(I * P, window, mode="same")

    varI = corrI - meanI * meanI
    covIP = corrIP - meanI * meanP
    a = covIP / (varI + e)
    b = meanP - a * meanI

    meana = sig.convolve2d(a, window, mode="same")
    meanb = sig.convolve2d(b, window, mode="same")

    q = meana * I + meanb
    
    Q = q.astype(float).tolist()
    
    # add metadata 
    Q.insert(0, img[1])
    Q.insert(0, img[0])
    return Q

def final_depth_map(data):
# Finds the restored depth map of the given image
# data: a 2D array representing the ppm of an image, incl. header metadata
# Returns a 2D array representing the final, restored depth pgm of an image, incl. header metadata
 
    # filter size for min filter:
    n = 15 
    # radius for guided filter:
    r = 8
    # regularization for edge detection
    eps = 0.004
    
    # raw depth map
    raw = raw_depth_map(data)
    #DEBUG
    scale_raw = scale_data(raw, 255)
    writepgm("scale_raw.pgm", scale_raw)
    # apply min/guided filters
    minn = min_filt(raw, n)
    gf = guided_filt(raw, minn, r, eps)
    # scale
    final = scale_data(gf, 1)
    return final

In [9]:
def restore_img(data, dmap):
    beta = 1.0

    w,h = data[0]
    img = []
    img.append(data[0])
    img.append(data[1])
    
    A = est_atm_light(dmap, data)
    A_arr = [A * w] * h
    print("Atmospheric Light: " + str(A))

    for i,r in enumerate(data[2:]):
        row = []
        for j,c in enumerate(r):
            # index in depth map
            jj = int(j/3)
            
            # find transmission, t(x)
            d_x = dmap[i+2][jj]
            e = math.e
            t = math.pow(e, (-beta*d_x))
            t_x = min(max(t, 0.1), 0.9)
            j_x = ((c - A_arr[i][j]) / t_x) + A_arr[i][j]
            row.append(j_x)
        img.append(row)
    
    return scale_data(img, 255)

In [10]:
# Test One: City

print("Reading image...")

city = readppm("images/hazy_city.ppm")

print("Recreating depth map...")

cityfdm = final_depth_map(city)

# print("Saving depth map...")

# writepgm("cityfdm.pgm", cityfdm)

print("Restoring...")

restore_city = restore_img(city, cityfdm) 

print("Saving restored image...")

writeppm("restore_city.ppm", restore_city)

Reading image...
Comment detected in images/hazy_city.ppm
Comment detected in images/hazy_city.ppm
Recreating depth map...
in scale_data: minv = -0.42692392998131934
in scale_data: maxv = 1.447509867811375
in scale_data: minv = -0.42692392998132
in scale_data: maxv = 1.1996916694312516
Restoring...
1.0
0.9998657866036369
0.9997718427736548
0.9993693625746283
0.9991812618634676
0.9987892029782482
0.9982284734279584
0.9980357562958201
0.9978913787766028
0.9978823855089822
0.9975585253511688
0.9975316630332877
0.9971405572082003
0.997071029153224
0.9970487272882844
0.9968836505363108
0.9964108906101794
0.996103362338131
0.9958668702176717
0.9958304418892687
0.9956863666832821
0.9956701649443896
0.9956413401833517
0.9955750102085233
0.9953729300646196
0.9952372847509685
0.9951969886861889
0.9950533475031299
0.9950436018940443
0.9948505935275134
0.9948153103636672
0.9946707099549849
0.9946527474353183
0.9945634353032469
0.9944844021001787
0.9943889090689988
0.9943625253327897
0.994179166135

in scale_data: minv = -52.07583560391987
in scale_data: maxv = 265.3453068958779
Saving restored image...


In [10]:
# Test Two: Canyon

print("Reading image...")

canyon = readppm("images/canyon.ppm")

print("Recreating depth map...")

canyonfdm = final_depth_map(canyon)

print("Saving depth map...")

writepgm("canyonfdm.pgm", canyonfdm)

print("Restoring...")

restore_canyon = restore_img(canyon,canyonfdm) 

print("Saving restored image...")

writeppm("restore_canyon.ppm", restore_canyon)


Reading image...
Recreating depth map...
in scale_data: minv = -0.7519085846336693
in scale_data: maxv = 1.9734657112381706
in scale_data: minv = -0.751908584633668
in scale_data: maxv = 1.7054183948626709
Saving depth map...
Restoring...
255
254
254
254
254
254
[(8, 672), (8, 671), (8, 670), (8, 669), (8, 668), (8, 667)]
(247, 247, 245)
in scale_data: minv = -2223.0
in scale_data: maxv = 2568.0
Saving restored image...


In [466]:
# Test Three: Nice

print("Reading image...")

nice = readppm("images/nice.ppm")

print("Recreating depth map...")

nicefdm = final_depth_map(nice)

print("Saving depth map...")

writepgm("nicefdm.pgm", nicefdm)

print("Restoring...")

restore_nice = restore_img(nice, nicefdm) 

print("Saving restored image...")

writeppm("restore_nice.ppm", restore_nice)

Reading image...
Recreating depth map...
in scale_data: minv = -0.7940531814408446
in scale_data: maxv = 1.699689336581156
Saving depth map...
Restoring...
in scale_data: minv = -585.0
in scale_data: maxv = 2676.0
Saving restored image...


In [447]:
# Test Four: Swiss

print("Reading image...")

swiss = readppm("images/swiss.ppm")

print("Recreating depth map...")

swissfdm = final_depth_map(swiss)

print("Saving depth map...")

writepgm("swissfdm.pgm", swissfdm)

print("Restoring...")

restore_swiss = restore_img(swiss, swissfdm) 

print("Saving restored image...")

writeppm("restore_swiss.ppm", restore_swiss)

Reading image...
Recreating depth map...
[[0.4806482873304595, 0.402980684276108, 0.46827839727205334, 0.4145203185149396, 0.37023949651122123, 0.39164797455808514, 0.4606411226322571, 0.345563477564463, 0.3461126590973781, 0.36267470695218723, 0.42827503686323454, 0.4386380235374359, 0.5329704221395977, 0.6062021549587066, 0.47619892218669, 0.4112312105750613, 0.42134154576915195, 0.47068945865647743, 0.47323477590714036, 0.5436400429941389, 0.5023227551670513, 0.4431178737162742, 0.48777064614751275, 0.406280756281186, 0.3122482884529976, 0.3459803456410352, 0.3462198128030532, 0.35564492413759263, 0.3553437092159922, 0.35425989630055105, 0.40195668824861347, 0.3932723787946387, 0.5243558233729192, 0.4197795366415928, 0.4067235573795326, 0.5067637684697356, 0.48763819104561823, 0.5408237417330065, 0.550433817994684, 0.4749444459137395, 0.4900695121556812, 0.5448729228895294, 0.4245297131563392, 0.47779926770027203, 0.5260562140252195, 0.39057646727908457, 0.43055191399396386, 0.36888

in scale_data: minv = -0.7861903452461761
in scale_data: maxv = 1.2615982741861307
Saving depth map...
Restoring...
in scale_data: minv = -24.44444444444443
in scale_data: maxv = 284.44444444444446
Saving restored image...


In [42]:
# Test Five: Plane

print("Reading image...")

plane = readppm("images/plane.ppm")

print("Recreating depth map...")

planefdm = final_depth_map(plane)

# print("Saving depth map...")

# writepgm("planefdm.pgm", planefdm)

print("Restoring...")

restore_plane = restore_img(plane, planefdm) 

print("Saving restored image...")

writeppm("restore_plane.ppm", restore_plane)

Reading image...
Recreating depth map...
in scale_data: minv = -0.5561470499644083
in scale_data: maxv = 1.8001095069262516
in scale_data: minv = -0.5561470499644087
in scale_data: maxv = 1.5352994094356145
Restoring...
1.0
0.9999605807904557
0.999935243327374
0.9999242005418292
[(19, 649), (19, 650), (18, 650), (20, 649)]
Atmospheric Light: (254, 254, 254)
in scale_data: minv = -101.72082204774057
in scale_data: maxv = 358.432191203017
Saving restored image...


In [498]:
# Test Six: Swans

print("Reading image...")

swans = readppm("images/swans.ppm")

print("Recreating depth map...")

swansfdm = final_depth_map(swans)

print("Saving depth map...")

writepgm("swansfdm.pgm", swansfdm)

print("Restoring...")

restore_swans = restore_img(swans, swansfdm) 

print("Saving restored image...")

writeppm("restore_swans.ppm", restore_swans)

Reading image...
Recreating depth map...
in scale_data: minv = -0.7530892793556969
in scale_data: maxv = 1.5626194694514746
Saving depth map...
Restoring...
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
254
253
253
253
253
253
253
253
253
253
253
253
253
253
253
253
253
253
253
253
253
253
253
253
253
253
253
253
253
253
253
253
253
253
253
253
253
253
253
253
253
253
253
253
253
253
253
253
253
253
253
253
253
253
253
253
253
253
253
253
253
253
253
253
253
253
253
253
253
253
253
253
253
253